### 由于在windows10下获取文件符号链接权限非常麻烦，所以另辟蹊径。采用shutil.move的方式对train、test文件夹进行处理
### 文件夹处理需要1分钟左右


In [11]:
root_src = 'D:\Dogs vs. Cats'
import os
import shutil
import time
os.chdir(root_src)



#创建训练数据所需的文件夹

def rmrf_mkdir(dirname,ctl='train'):
    start = time.time()
    if os.path.exists(dirname):
        pass
    else:
        if ctl=='train':
            os.mkdir(dirname)
            os.mkdir('train2/cat')
            os.mkdir('train2/dog')
            #获取当前狗和猫图片的文件名
            train_filenames = os.listdir('train')
            train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
            train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)
            #创建train2文件夹，并将cat类和dog类图片归类到不同的文件夹中
            for filename in train_cat:
                shutil.move('./train/'+filename, './train2/cat/'+filename)

            for filename in train_dog:
                shutil.move('./train/'+filename, './train2/dog/'+filename)
        elif ctl=='test':
            test_filenames = os.listdir(dirname.split('/')[0])
            os.mkdir(dirname)
            for filename in test_filenames:
                shutil.move('test/'+filename,dirname+'/'+filename)
        else:
            pass
    end = time.time()
    print('path deal time is %.5f'%round(end-start,5))

rmrf_mkdir('train2',ctl='train')  
rmrf_mkdir('test/test2',ctl='test')



path deal time is 34.39030
path deal time is 17.12331


### 通过预处理获得了ResNet50，InceptionV3，Xception三个模型的的特征参数,三个模型在GTX1070下跑获取参数时间大约花费40分钟。

In [6]:

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import numpy as np
import h5py
def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("train2", image_size, shuffle=False, 
                                              batch_size=16)
    test_generator = gen.flow_from_directory("test", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)
    train = model.predict_generator(train_generator, len(train_generator.filenames)/16)
    test = model.predict_generator(test_generator, len(test_generator.filenames)/16)
    print(type(train),train.shape) 
    print(type(test),test.shape) 
    with h5py.File("gap_%s.h5"%(MODEL.__name__)) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)
start = time.time()  
write_gap(ResNet50, (224, 224))
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)
write_gap(Xception, (299, 299), xception.preprocess_input)
end = time.time()
print('one generator time is %d s'%round(end-start))

Using TensorFlow backend.


Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
<class 'numpy.ndarray'> (25000, 2048)
<class 'numpy.ndarray'> (12500, 2048)
Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
<class 'numpy.ndarray'> (25000, 2048)
<class 'numpy.ndarray'> (12500, 2048)
Found 25000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
<class 'numpy.ndarray'> (25000, 2048)
<class 'numpy.ndarray'> (12500, 2048)
one generator time is 2341 s
